In [89]:
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.models import load_model

In [90]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
tf.keras.utils.plot_model(model, show_shapes=True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [97]:
train_path=r'C:\Users\nyunu\Desktop\Mydata\Training'
test_path=r'C:\Users\nyunu\Desktop\Mydata\Testing'

In [98]:
x_train=ImageDataGenerator(rescale=1./255).flow_from_directory(train_path,batch_size=32,classes=['mask','no_mask'],class_mode='categorical',target_size=(100,100),shuffle=True)
x_test=ImageDataGenerator(rescale=1./255).flow_from_directory(test_path,batch_size=32,classes=['mask','no_mask'],class_mode='categorical',target_size=(100,100),shuffle=True)

Found 5772 images belonging to 2 classes.
Found 1133 images belonging to 2 classes.


In [99]:
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.models import Sequential

In [100]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same',input_shape=(100 , 100 , 3)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax')) 
tf.keras.utils.plot_model(model, show_shapes=True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [108]:
model.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.fit_generator(x_train, epochs=2,steps_per_epoch=10,verbose=1,validation_data=x_test,validation_steps=5)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 10 steps, validate for 5 steps
Epoch 1/2
10/10 [==============================] - 13s 1s/step - loss: 0.4959 - accuracy: 0.8188 - val_loss: 0.4920 - val_accuracy: 0.8125
Epoch 2/2
10/10 [==============================] - 13s 1s/step - loss: 0.4848 - accuracy: 0.8313 - val_loss: 0.5342 - val_accuracy: 0.7688


In [109]:
model.save(r'C:\Users\nyunu\Desktop\Mydata')

INFO:tensorflow:Assets written to: C:\Users\nyunu\Desktop\Mydata\assets


In [111]:
model=load_model(r'C:\Users\nyunu\Desktop\Mydata')

In [104]:
face_classifier=cv2.CascadeClassifier(r'C:\Users\nyunu\Desktop\Real-Time-Image-Classification-master\haarcascade_frontalface_default.xml')

In [112]:
def predict(model,cap):
    prediction=model.predict(cap)
    return np.argmax(prediction)

In [115]:
vid=cv2.VideoCapture(0)
if not vid.isOpened():
    print('cannot open the camera')
    exit()
while True:
    ret,cap=vid.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    gray=cv2.cvtColor(cap,cv2.COLOR_BGR2GRAY)
    face=face_classifier.detectMultiScale(gray, 1.05, 5)

    for (x,y,a,b) in face:
        cv2.rectangle(cap,(x,y),(x+a,y+b),(127,0,255),2)
        roi_color=cap[y:y+b,x:x+a]
        img = cv2.resize(roi_color, (100,100), interpolation=cv2.INTER_CUBIC)
        img = img.astype('float16')
        img = np.expand_dims(img, axis=0)
        score=predict(model,img)
        if score==1:
            cv2.putText(cap,'Mask',(x+a,y+b), cv2.FONT_HERSHEY_SIMPLEX ,0.5,(255,0,0),1, cv2.LINE_AA)  
          
        if score==0:
            cv2.putText(cap,'No Mask',(x+a,y+b), cv2.FONT_HERSHEY_SIMPLEX ,0.5,(255,0,0),1, cv2.LINE_AA)
           
        
        cv2.imshow('image',cap)
        cv2.waitKey(2)
    if cv2.waitKey(2) & 0xFF==ord('q'):
        break
vid.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 